In [126]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
import string
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import precision_score, recall_score, auc, accuracy_score, roc_auc_score, f1_score
from sklearn.metrics import classification_report, roc_curve
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ivan2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivan2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
df = pd.read_csv('labeled.csv')

In [40]:
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [41]:
df.shape

(14412, 2)

In [42]:
df['toxic'] = df['toxic'].apply(int)

In [43]:
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1
2,Собаке - собачья смерть\n,1
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1


In [44]:
df['toxic'].value_counts()

toxic
0    9586
1    4826
Name: count, dtype: int64

In [45]:
for i in df[df['toxic'] == 0]['comment'].head(5):
    print(i)

В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.

Почитайте посты у этого автора,может найдете что нибудь полезное. Надеюсь помог) https: pikabu.ru story obyichnyie budni dezsluzhbyi 4932098

Про графику было обидно) я так то проходил все серии гта со второй части по пятую, кроме гта 4. И мне не мешала графика ни в одной из частей. На компе у меня было куча видеокарт. Начиная с 32мб RIVA TNT и заканчивая 2Гб 560Ti на которой я спокойно играю который год в танки, гта5, ведьмака3 купил на распродаже и начал проходить. Да, не на ультрах. С пониженными текстурами. И не мешает. Я не понимаю дрочева на графике, требовать графику уровня плойки 4 минимум. Мне надо чтобы глаза не резало, только и всего. По поводу управления, мне не хватает переходника

In [46]:
X = df['comment']
y = df['toxic']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

In [48]:
X_test.value_counts()

comment
В майнкрафт разве что, угу... А так, нынче нет игр для них особо, да и, честно говоря, у меня не шибко накачанная шея, чтобы наклоняться долго (все-таки, он не просто для сидения за компом - он удобен в движении).\n                                                                                                                   1
Либертарианство технически невыполнимо в масштабах человечества, потому что не подходит большинству Но оно как раз существует в масштабах человечества, т.к. нет власти над государствами. А ниже все люди объединились своими контрактами. Для полного либертарианского совершенства социальные контракты нужно оформить письменно.\n    1
Виодук это не ошибка.\n                                                                                                                                                                                                                                                                                                                 

In [49]:
X_train.value_counts()

comment
А в горшочке ,да в печи,картофан,да в мундире.эээх\n                                                                                                                                                                                                                                                                                                                                                                                               1
Гений,миллиардер,филантроп,пиздабол\n                                                                                                                                                                                                                                                                                                                                                                                                              1
А там я ел перловку. 2к16 год. Правда бигусом там любую капусту называли. Мне нравилось. А пайки давали когда кухня сб

In [50]:
df['toxic'].value_counts()

toxic
0    9586
1    4826
Name: count, dtype: int64

In [51]:
sentence_example = df.iloc[1]['comment']

In [52]:
sentence_example

'Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n'

In [53]:
tokens = word_tokenize(sentence_example, language='russian')

In [54]:
tokens

['Хохлы',
 ',',
 'это',
 'отдушина',
 'затюканого',
 'россиянина',
 ',',
 'мол',
 ',',
 'вон',
 ',',
 'а',
 'у',
 'хохлов',
 'еще',
 'хуже',
 '.',
 'Если',
 'бы',
 'хохлов',
 'не',
 'было',
 ',',
 'кисель',
 'их',
 'бы',
 'придумал',
 '.']

In [55]:
tokens_without_punkt = [i for i in tokens if i not in string.punctuation]

In [56]:
russian_stopwords = stopwords.words('russian')


In [57]:
tokens_without_p_and_sw = [i for i in tokens_without_punkt if i not in russian_stopwords]

In [58]:
snowball = SnowballStemmer(language='russian')

In [59]:
tokens = [snowball.stem(i) for i in tokens_without_p_and_sw]

In [60]:
tokens

['хохл',
 'эт',
 'отдушин',
 'затюкан',
 'россиянин',
 'мол',
 'вон',
 'хохл',
 'хуж',
 'есл',
 'хохл',
 'кисел',
 'придума']

In [61]:
type(russian_stopwords)

list

In [62]:
snwoball = SnowballStemmer(language='russian')
russian_sw = stopwords.words('russian')

def tokenize(word: str, remove_stop_words: bool = True):
    tokens = word_tokenize(word, language='russian')
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_sw]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [104]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize(x, remove_stop_words=True))

In [103]:
X_train

12270    А в горшочке ,да в печи,картофан,да в мундире....
4288                             Соблазнюшка! Пойду поем\n
11282    Как раз таки это нормально, чтобы конкуренты н...
4663     Там ведь ещё год два назад пенсионеров стали с...
9499     Хмм, действительно!) Здравая мысль. Не чаще? Г...
                               ...                        
10206    Пост для разьяснения почему билеты на ж д доро...
6253     А в рашке традиция: или вперёд ногами из Кремл...
10123    Картинка прямо 1 в 1 из моего сна. Часто такие...
5600     Вот и хохлы подгоревшие подъехали. Куда Ньюсач...
14000    РФ - продолжение СССР, тут всегда будут тихо н...
Name: comment, Length: 10088, dtype: object

In [105]:
features = vectorizer.fit_transform(X_train)

C:\Users\ivan2\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [89]:
features

<10088x29462 sparse matrix of type '<class 'numpy.float64'>'
	with 157249 stored elements in Compressed Sparse Row format>

In [107]:
model = LogisticRegression(random_state=0)
model.fit(features, y_train)

LogisticRegression(random_state=0)

In [68]:
model.predict(features[0])

array([1], dtype=int64)

In [95]:
X_train.iloc[0]

'А в горшочке ,да в печи,картофан,да в мундире.эээх\n'

In [93]:
model.predict(features[0])

array([1], dtype=int64)

In [70]:
model_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer=lambda x: tokenize(x, remove_stop_words=True))),
    ('model', LogisticRegression(random_state=0))
]
)



In [111]:
model_pipeline.fit(X_train, y_train)
y_pred = model_pipeline.predict(X_test)

C:\Users\ivan2\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [123]:
model_pipeline.predict(["раз два три"])

array([0], dtype=int64)

In [112]:
y_pred

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [116]:
print('recall', recall_score(y_test, y_pred))
print('precission', precision_score(y_test, y_pred))

recall 0.6117566643882434
precission 0.9086294416243654
